In [4]:
import json
import os
import csv
import pandas as pd


In [5]:
def transform_participant_data(participant_data):
    camel_scores = [0] * 5
    camel_times = [0] * 5
    kebab_scores = [0] * 5
    kebab_times = [0] * 5

    # Load questionData dictionary
    question_data_dict = participant_data["questionData"]

    for _, question_data in question_data_dict.items():
        case_format = question_data["caseFormat"]
        is_correct = 1 if question_data["isCorrect"] else 0
        time_taken = question_data["timeTaken"]

        if case_format.startswith("camel"):
            camel_scores[int(case_format[-1]) - 1] = is_correct
            camel_times[int(case_format[-1]) - 1] = time_taken
        elif case_format.startswith("kebab"):
            kebab_scores[int(case_format[-1]) - 1] = is_correct
            kebab_times[int(case_format[-1]) - 1] = time_taken

    camel_average = sum(camel_times) / len(camel_times) if any(camel_times) else 0
    kebab_average = sum(kebab_times) / len(kebab_times) if any(kebab_times) else 0

    return {
        "participantID": participant_data["participantID"],
        "experience": participant_data["demographicsAnswers"][0]["experience"],
        "camel1_score": camel_scores[0],
        "camel2_score": camel_scores[1],
        "camel3_score": camel_scores[2],
        "camel4_score": camel_scores[3],
        "camel5_score": camel_scores[4],
        "camel1_time": camel_times[0],
        "camel2_time": camel_times[1],
        "camel3_time": camel_times[2],
        "camel4_time": camel_times[3],
        "camel5_time": camel_times[4],
        "camel_average": camel_average,
        "kebab1_score": kebab_scores[0],
        "kebab2_score": kebab_scores[1],
        "kebab3_score": kebab_scores[2],
        "kebab4_score": kebab_scores[3],
        "kebab5_score": kebab_scores[4],
        "kebab1_time": kebab_times[0],
        "kebab2_time": kebab_times[1],
        "kebab3_time": kebab_times[2],
        "kebab4_time": kebab_times[3],
        "kebab5_time": kebab_times[4],
        "kebab_average": kebab_average,
    }


In [6]:
# List to store transformed data for all participants
all_participants_data_transformed = []

directory_path = './data/'

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)

        # Load data from the file
        with open(file_path, 'r') as file:
            data = json.load(file)

        # Transform data for each participant in the file
        participants_data_transformed = [transform_participant_data(data)]

        # Append transformed data to the list
        all_participants_data_transformed.extend(participants_data_transformed)

# Create a DataFrame from the transformed data
df = pd.DataFrame(all_participants_data_transformed)

# Display the DataFrame
df.head()  # You can adjust the number of rows displayed by changing the argument to head()

# CSV file path
csv_file_path = "./output.csv"

# Write data to CSV file
df.to_csv(csv_file_path, index=False)

print(f"Data has been saved to {csv_file_path}")

# Display the CSV data as a string
csv_data_string = df.to_string(index=False)

# Print the CSV data string
print(csv_data_string)

Data has been saved to ./output.csv
                       participantID    experience  camel1_score  camel2_score  camel3_score  camel4_score  camel5_score  camel1_time  camel2_time  camel3_time  camel4_time  camel5_time  camel_average  kebab1_score  kebab2_score  kebab3_score  kebab4_score  kebab5_score  kebab1_time  kebab2_time  kebab3_time  kebab4_time  kebab5_time  kebab_average
f016b9fd-1799-4b3b-a8a4-8b4f0dd87e26 no-experience             1             1             1             0             1       7248.9      29147.2      13586.3     248251.5      11786.9       62004.16             1             1             1             1             1       7056.1      13175.3  8182.600000      15760.3      11522.7   11139.400000
d3d7734f-c406-460f-bf3b-4e2d405bc066  bachelor-inf             1             0             1             1             1       6152.0       9084.9       8206.0       6248.2       5848.6        7107.94             1             1             1             1      